In [75]:
import importlib
from IPython.display import display
import pandas as pd
import numpy as np
import account as acc

In [30]:
importlib.reload(acc)

<module 'account' from '/Users/KP_Hong/Documents/PyPrjt/[lib]PrjtCashFlowModel_v3/PrjtCF_module/account.py'>

In [31]:
#### INPUT BASIC DATA ####
mtrt = 26 #개월
prd_prjt = 24 #개월(프로젝트 기간)
repay_rate = 1.0 #상환비율(매출 발생금액 중 상환계좌 이체 비율)
#### INPUT BASIC DATA ####

In [32]:
#### INPUT SALES DATA ####
# Initial sales data
# Product A : 상온창고
prdtA_rent = 21_000 # 월 임대료 21,000원/평 가정
prdtA_area = 8_000 # 면적 8,000평
prdtA_cap = 0.053 # Cap rate 5.3%
prdtA_amt = prdtA_rent * 12 * prdtA_area / prdtA_cap / 1_000_000
prdtA = acc.account(title="ProductA", tag=("ProductA", "-"), mtrt=mtrt,
                         add_scdd_ipt=pd.Series([prdtA_amt], index=[0]),
                         sub_scdd_ipt=pd.Series(np.array([1.0])* prdtA_amt, index=[24]),
                         note="-")
prdtA.addamt(0, prdtA.add_rsdl(0))


# Product B : 저온창고
prdtB_rent = 50_000 # 월 임대료 50,000원/평 가정
prdtB_area = 6_500 # 면적 6_500평
prdtB_cap = 0.058 # Cap rate 5.8%
prdtB_amt = prdtB_rent * 12 * prdtB_area / prdtB_cap / 1_000_000
prdtB = acc.account(title="ProductB", tag=("ProductB", "-"), mtrt=mtrt,
                         add_scdd_ipt=pd.Series([prdtB_amt], index=[0]),
                         sub_scdd_ipt=pd.Series(np.array([1.0])* prdtB_amt, index=[24]),
                         note="-")
prdtB.addamt(0, prdtB.add_rsdl(0))

sales = acc.merge({'prdtA':prdtA, 'prdtB':prdtB})

# Scenario of sales
sales.dct['prdtA'].sub_exctn([24], [1.0])
sales.dct['prdtB'].sub_exctn([24], [1.0])
sales_rate = (sales.df.amt_sub / sales.df.add_scdd_cum).replace(np.nan, 0.0)
#### INPUT SALES DATA ####

In [35]:
#### INPUT COST DATA ####
# Scenario of cost
prcs_rate = pd.Series(np.ones(prd_prjt) * 1/prd_prjt, index=np.arange(1, prd_prjt+1)) # 공정률

# Initial cost data
cost_lnd_amt = 15_500.000 # 최초 1회 지급
cost_lnd = acc.account(title="토지비", tag=("토지비", "-"), mtrt=mtrt,
                    add_scdd_ipt = pd.Series([cost_lnd_amt], index=[0]),
                    note="-")

cost_cstrn_amt = 46_000.000 # 공정률에 따라 지급
cost_cstrn_prcs = acc.account(title="직접공사비", tag=("공사비", "-"), mtrt=mtrt,
                    add_scdd_ipt = cost_cstrn_amt * 0.8 * prcs_rate, # 공사비 기성 지급 80%
                    note="-")
cost_cstrn_rsrv = acc.account(title="직접공사비", tag=("공사비", "-"), mtrt=mtrt,
                    add_scdd_ipt = pd.Series([cost_cstrn_amt * 0.2], index=[prd_prjt]), # 공사비 지급유보 20%
                    note="-")

cost_cstrnb_amt = 3_000.000 # 최초 1회 지급
cost_cstrnb = acc.account(title="간접공사비", tag=("공사비", "-"), mtrt=mtrt,
                    add_scdd_ipt = pd.Series([cost_cstrnb_amt], index=[0]),
                    note="-")

cost_mrktg_amt = 2_000.000 # 공정률에 따라 지급
cost_mrktg = acc.account(title="홍보비용", tag=("홍보비", "-"), mtrt=mtrt,
                    add_scdd_ipt = cost_mrktg_amt * prcs_rate,
                    note="-")

cost_tax_amt = 3_000.000 # 준공시 1회 지급
cost_tax = acc.account(title="제세공과금", tag=("제세공과금", "-"), mtrt=mtrt,
                    add_scdd_ipt = pd.Series([cost_tax_amt], index=[prd_prjt]),
                    note="-")

cost_oprtg_amt = 2_200.000 # 공정률에 따라 지급
cost_oprtg = acc.account(title="신탁운영비", tag=("운영비", "-"), mtrt=mtrt,
                    add_scdd_ipt = cost_oprtg_amt * prcs_rate,
                    note="-")

dct = {'cost_lnd':cost_lnd, 'cost_cstrn_prcs':cost_cstrn_prcs, 'cost_cstrn_rsrv':cost_cstrn_rsrv, 
       'cost_cstrnb':cost_cstrnb, 'cost_mrktg':cost_mrktg, 'cost_tax':cost_tax, 'cost_oprtg':cost_oprtg}
cost = acc.merge(dct)
#### INPUT COST DATA ####

In [36]:
#### INPUT LOAN DATA ####
loanA = acc.Loan(title="loanA", tag=("loanA", "-"), mtrt=mtrt,
                 ntnl=55_000.000, rate=0.045, note="-")
loanA_fee = acc.account(title="loanA_fee", tag=("취급수수료", "-"), mtrt=mtrt,
                       add_scdd_ipt = pd.Series([loanA.ntnl * 0.01], index=[0]))
loanB = acc.Loan(title="loanB", tag=("loanB", "-"), mtrt=mtrt,
                 ntnl=20_000.000, rate=0.07, note="-")
loanB_fee = acc.account(title="loanB_fee", tag=("취급수수료", "-"), mtrt=mtrt,
                       add_scdd_ipt = pd.Series([loanB.ntnl * 0.06], index=[0]))
loan_arng = acc.account(title="loan_arng", tag=("주선수수료", "-"), mtrt=mtrt,
                       add_scdd_ipt = pd.Series([(loanA.ntnl + loanB.ntnl) * 0.015], index=[0]))

dct = {'loanA_fee':loanA_fee, 'loanB_fee':loanB_fee, 'loan_arng':loan_arng}
loan_fee = acc.merge(dct)
#### INPUT LOAN DATA ####

In [37]:
#### CASH FLOW ####
# Make accounts
acc_repay = acc.account(title="Repayment Account", tag=("account", "repayment"), mtrt=mtrt, note="")
acc_oprtg = acc.account(title="Operating Account", tag=("account", "operating"), mtrt=mtrt, note="")
# acc_ovrdrft = acc.account(title="Overdraft Account", tag=("account", "overdraft"), mtrt=mtrt, note="")

# Make Cash Flow
for idx in acc_oprtg.df.index:
    #### Sales generation ####
    sales_amt = sales.df.loc[idx, 'amt_sub']
    sales_rate = sales_amt / sales.df.loc[idx, 'add_scdd_cum'] if sales.df.loc[idx, 'add_scdd_cum'] != 0 else 0.0
    
    acc_repay.addamt(idx, sales_amt * repay_rate, note="repay_rate of sales_amt")
    acc_oprtg.addamt(idx, sales_amt * (1 - repay_rate), note="pay_rate of sales_amt")
    #### Sales generation ####
    
    #### Transfer amount on repayment account to operating account ####
    amt_trsfr_A = min(acc_repay.bal_end(idx), loanA.amt.df.loc[idx, 'sub_rsdl_cum'])
                  # min(balance on repayment account, amt withdrawable)
    acc_repay.subamt(idx, amt_trsfr_A, note="transfer to operating account") # repayment account decrease
    acc_oprtg.addamt(idx, amt_trsfr_A, note="transfered from repayment account") # operating account increase
    loanA.amt.subscdd(idx, -amt_trsfr_A) # reduce scheduled loan amount
    
    if loanA.is_repaid(idx) is True:
        amt_trsfr_B = min(acc_repay.bal_end(idx), loanB.amt.df.loc[idx, 'sub_rsdl_cum']) 
                      # min(balance on repayment account, amt withdrawable)
        acc_repay.subamt(idx, amt_trsfr_B, note="transfer to operating account") # repayment account decrease
        acc_oprtg.addamt(idx, amt_trsfr_B, note="transfered from repayment account") # operating account increase
        loanB.amt.subscdd(idx, -amt_trsfr_B) # reduce scheduled loan amount
        
        if loanB.is_repaid(idx) is True:
            amt_trsfr_bal = acc_repay.bal_end(idx)
            acc_repay.subamt(idx, amt_trsfr_bal, note="transfer balance to operating account")
            acc_oprtg.addamt(idx, amt_trsfr_bal, note="transfer balance of repayment account")
    #### Transfer amount on repayment account to operating account ####
    
    #### Estimate financial cost amount ####
    # IR to pay amount
    IR_amt_A = loanA.amt.amt_sub_cum(idx) * loanA.rate / 12
    IR_amt_B = loanB.amt.amt_sub_cum(idx) * loanB.rate / 12
    loanA.IR.addscdd(idx, IR_amt_A)
    loanB.IR.addscdd(idx, IR_amt_B)
    IR_amt_ttl = IR_amt_A + IR_amt_B
    
    loanA_fee = loan_fee.dct['loanA_fee'].add_rsdl(idx)
    loanB_fee = loan_fee.dct['loanB_fee'].add_rsdl(idx)
    loan_arng = loan_fee.dct['loan_arng'].add_rsdl(idx)
    loanfee_amt_ttl = loanA_fee + loanB_fee + loan_arng
    #### Estimate financial cost amount ####
    
    #### Estimate operating cost amount ####
    cost_amt_ttl = sum([cost.dct[x].add_rsdl(idx) for x in cost.dct])
    #costA_tmp = cost.dct['costA'].add_rsdl(idx)
    #costB_tmp = cost.dct['costB'].add_rsdl(idx)
    #costC_tmp = cost.dct['costC'].add_rsdl(idx)
    #cost_amt_ttl = costA_tmp + costB_tmp + costC_tmp
    #### Estimate operating cost amount ####
    
    #### Estimate short to pay and pay loan ####
    ttlamt_topay = IR_amt_ttl + loanfee_amt_ttl + cost_amt_ttl # amount to pay
    short_amt = ttlamt_topay - acc_oprtg.bal_end(idx) # amount to short
    
    tmpamt_rqrd = 0
    if short_amt > 0:
        payamt_min = 1_000
        tmpamt_rqrd = np.ceil(short_amt / payamt_min) * payamt_min
        
        # withdraw Tr.B loan amount
        tmpamt_topay = min(loanB.amt.df.loc[idx, 'sub_rsdl_cum'], tmpamt_rqrd)
        tmpamt_rqrd -= tmpamt_topay
        loanB.amt.subamt(idx, tmpamt_topay, note="withdraw loan B") # withdraw loan B
        acc_oprtg.addamt(idx, tmpamt_topay, note="withdraw loan B") # transfer to operating account
        
        # withdraw Tr.A loan amount
        tmpamt_topay = min(loanA.amt.df.loc[idx, 'sub_rsdl_cum'], tmpamt_rqrd)
        tmpamt_rqrd -= tmpamt_topay
        loanA.amt.subamt(idx, tmpamt_topay, note="withdraw loan A") # withdraw loan A
        acc_oprtg.addamt(idx, tmpamt_topay, note="withdraw loan A") # transfer to operating account
    #### Estimate short to pay and pay loan ####
    
    #### Pay financial cost ####
    amt_pay_feeA = loan_fee.dct['loanA_fee'].add_rsdl(idx)
    acc_oprtg.subamt(idx, amt_pay_feeA, note="pay fee on loan A")
    loan_fee.dct['loanA_fee'].addamt(idx, amt_pay_feeA)

    amt_pay_feeB = loan_fee.dct['loanB_fee'].add_rsdl(idx)
    acc_oprtg.subamt(idx, amt_pay_feeB, note="pay fee on loan B")
    loan_fee.dct['loanB_fee'].addamt(idx, amt_pay_feeB)

    amt_pay_arng = loan_fee.dct['loan_arng'].add_rsdl(idx)
    acc_oprtg.subamt(idx, amt_pay_arng, note="pay fee on arangement")
    loan_fee.dct['loan_arng'].addamt(idx, amt_pay_arng)
    
    amt_repay_A = loanA.IR.add_rsdl(idx)
    acc_oprtg.subamt(idx, amt_repay_A, note="repay IR on loan A")
    loanA.IR.addamt(idx, amt_repay_A)
    
    amt_repay_B = loanB.IR.add_rsdl(idx)
    acc_oprtg.subamt(idx, amt_repay_B, note="repay IR on loan B")
    loanB.IR.addamt(idx, amt_repay_B)
    #### Pay financial cost ####
    
    #### Pay operating cost ####
    for key, val in cost.dct.items():
        amt_pay = val.add_rsdl(idx)
        acc_oprtg.subamt(idx, amt_pay, note="pay cost")
        cost.dct[key].addamt(idx, amt_pay)
    #### Pay operating cost ####
    
    #### Repay notional amount ####
    # repay loan from repayment account
    loan_repay_A = min(acc_repay.bal_end(idx), loanA.loan_rsdl(idx))
    acc_repay.subamt(idx, loan_repay_A, note="repay loan A from repayment account")
    loanA.amt.addamt(idx, loan_repay_A, note="repay loan from repayment account")
    
    if loanA.is_repaid(idx) is True:
        # repay loan B from repayment account
        loan_repay_B = min(acc_repay.bal_end(idx), loanB.loan_rsdl(idx))
        acc_repay.subamt(idx, loan_repay_B, note="repay loan B from repayment account")
        loanB.amt.addamt(idx, loan_repay_B, note="repay loan from repayment account")
    
    # repay loan from operating account
    repayamt_min = 10_000
    if acc_oprtg.bal_end(idx) > repayamt_min:
        # repay loan A from operating account
        loan_repay_A = min(acc_oprtg.bal_end(idx), loanA.loan_rsdl(idx))
        acc_oprtg.subamt(idx, loan_repay_A, note="repay loan A from operating account")
        loanA.amt.addamt(idx, loan_repay_A, note="repay loan from operating account")
        
        if loanA.is_repaid(idx) is True:
            # repay loan B from operating account
            loan_repay_B = min(acc_oprtg.bal_end(idx), loanB.loan_rsdl(idx))
            acc_oprtg.subamt(idx, loan_repay_B, note="repay loan B from operating account")
            loanB.amt.addamt(idx, loan_repay_B, note="repay loan from operating account")
    
    #### Repay notional amount ####
#### CASH FLOW ####

In [76]:
#### Print Data ####
columns = [['acc_oprtg', 'acc_oprtg', 'acc_oprtg', 'acc_oprtg', 
            'acc_repay', 'acc_repay', 'acc_repay', 'acc_repay', 
            'sales', 'sales', 'cost', 'cost', 
            'loanA.amt', 'loanA.amt', 'loanA.amt', 'loanB.amt', 'loanB.amt', 'loanB.amt', 
            'loanA.IR', 'loanA.IR', 'loanB.IR', 'loanB.IR', 'loan_fee'], 
 ['bal_strt', 'amt_add', 'amt_sub', 'bal_end', 
  'bal_strt', 'amt_add', 'amt_sub', 'bal_end', 
  'amt_sub', 'bal_end', 'amt_add', 'bal_end', 
  'amt_sub', 'amt_sub_cum', 'bal_end', 'amt_sub', 'amt_sub_cum', 'bal_end', 
  'amt_add', 'amt_add_cum', 'amt_add', 'amt_add_cum', 'amt_add']]
result = pd.concat([acc_oprtg.df['bal_strt'], acc_oprtg.df['amt_add'], acc_oprtg.df['amt_sub'], acc_oprtg.df['bal_end'],
                   acc_repay.df['bal_strt'], acc_repay.df['amt_add'], acc_repay.df['amt_sub'], acc_repay.df['bal_end'],
                   sales.df['amt_sub'], sales.df['bal_end'], cost.df['amt_add'], cost.df['bal_end'],
                   loanA.amt.df['amt_sub'], loanA.amt.df['amt_sub_cum'], loanA.amt.df['bal_end'],
                   loanB.amt.df['amt_sub'], loanB.amt.df['amt_sub_cum'], loanB.amt.df['bal_end'],
                   loanA.IR.df['amt_add'], loanA.IR.df['amt_add_cum'],
                   loanB.IR.df['amt_add'], loanB.IR.df['amt_add_cum'], loan_fee.df['amt_add']
                   ], axis=1)
result.columns = columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(result)
#### Print Data ####

acc_oprtg                                               acc_repay  \
         bal_strt       amt_add       amt_sub       bal_end      bal_strt   
-1       0.000000      0.000000      0.000000      0.000000      0.000000   
 0       0.000000  22000.000000  21375.000000    625.000000      0.000000   
 1     625.000000   2000.000000   1832.500000    792.500000      0.000000   
 2     792.500000   2000.000000   1840.000000    952.500000      0.000000   
 3     952.500000   1000.000000   1847.500000    105.000000      0.000000   
 4     105.000000   2000.000000   1851.250000    253.750000      0.000000   
 5     253.750000   2000.000000   1858.750000    395.000000      0.000000   
 6     395.000000   2000.000000   1866.250000    528.750000      0.000000   
 7     528.750000   2000.000000   1873.750000    655.000000      0.000000   
 8     655.000000   2000.000000   1881.250000    773.750000      0.000000   
 9     773.750000   2000.000000   1888.750000    885.000000      0.000000   
 10    885.000000   2000.000000   1896.250000    988.750000      0.000000   
 11    988.750000   1000.000000   1903.750000     85.000000      0.000000   
 12     85.000000   2000.000000   1907.500000    177.500000      0.000000   
 13    177.500000   2000.000000   1915.000000    262.500000      0.000000   
 14    262.500000   2000.000000   1922.500000    340.000000      0.000000   
 15    340.000000   2000.000000   1930.000000    410.000000      0.000000   
 16    410.000000   2000.000000   1937.500000    472.500000      0.000000   
 17    472.500000   2000.000000   1945.000000    527.500000      0.000000   
 18    527.500000   2000.000000   1952.500000    575.000000      0.000000   
 19    575.000000   2000.000000   1960.000000    615.000000      0.000000   
 20    615.000000   2000.000000   1967.500000    647.500000      0.000000   
 21    647.500000   2000.000000   1975.000000    672.500000      0.000000   
 22    672.500000   2000.000000   1982.500000    690.000000      0.000000   
 23    690.000000   2000.000000   1990.000000    700.000000      0.000000   
 24    700.000000   9000.000000  14197.500000  -4497.500000      0.000000   
 25  -4497.500000  30279.115159    289.166667  25492.448493  30279.115159   
 26  25492.448493      0.000000    289.166667  25203.281826      0.000000   

                                                        sales                 \
           amt_add       amt_sub       bal_end        amt_sub        bal_end   
-1        0.000000      0.000000      0.000000       0.000000       0.000000   
 0        0.000000      0.000000      0.000000       0.000000  105279.115159   
 1        0.000000      0.000000      0.000000       0.000000  105279.115159   
 2        0.000000      0.000000      0.000000       0.000000  105279.115159   
 3        0.000000      0.000000      0.000000       0.000000  105279.115159   
 4        0.000000      0.000000      0.000000       0.000000  105279.115159   
 5        0.000000      0.000000      0.000000       0.000000  105279.115159   
 6        0.000000      0.000000      0.000000       0.000000  105279.115159   
 7        0.000000      0.000000      0.000000       0.000000  105279.115159   
 8        0.000000      0.000000      0.000000       0.000000  105279.115159   
 9        0.000000      0.000000      0.000000       0.000000  105279.115159   
 10       0.000000      0.000000      0.000000       0.000000  105279.115159   
 11       0.000000      0.000000      0.000000       0.000000  105279.115159   
 12       0.000000      0.000000      0.000000       0.000000  105279.115159   
 13       0.000000      0.000000      0.000000       0.000000  105279.115159   
 14       0.000000      0.000000      0.000000       0.000000  105279.115159   
 15       0.000000      0.000000      0.000000       0.000000  105279.115159   
 16       0.000000      0.000000      0.000000       0.000000  105279.115159   
 17       0.000000      0.000000      0.000000       0.000000  105279.115159   
 18       0.00000

In [61]:
acc_oprtg.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,0.0,0.0,0.0,0.0,0.000000,22000.000000,22000.000000,21375.000000,21375.000000,625.000000,-22000.000000,-21375.000000
1,0.0,0.0,0.0,0.0,625.000000,2000.000000,24000.000000,1832.500000,23207.500000,792.500000,-24000.000000,-23207.500000
2,0.0,0.0,0.0,0.0,792.500000,2000.000000,26000.000000,1840.000000,25047.500000,952.500000,-26000.000000,-25047.500000
3,0.0,0.0,0.0,0.0,952.500000,1000.000000,27000.000000,1847.500000,26895.000000,105.000000,-27000.000000,-26895.000000
4,0.0,0.0,0.0,0.0,105.000000,2000.000000,29000.000000,1851.250000,28746.250000,253.750000,-29000.000000,-28746.250000
5,0.0,0.0,0.0,0.0,253.750000,2000.000000,31000.000000,1858.750000,30605.000000,395.000000,-31000.000000,-30605.000000
6,0.0,0.0,0.0,0.0,395.000000,2000.000000,33000.000000,1866.250000,32471.250000,528.750000,-33000.000000,-32471.250000
7,0.0,0.0,0.0,0.0,528.750000,2000.000000,35000.000000,1873.750000,34345.000000,655.000000,-35000.000000,-34345.000000
8,0.0,0.0,0.0,0.0,655.000000,2000.000000,37000.000000,1881.250000,36226.250000,773.750000,-37000.000000,-36226.250000


In [39]:
acc_repay.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
sales.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
0,105279.115159,105279.115159,0.000000,0.000000,0.000000,105279.115159,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
1,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
2,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
3,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
4,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
5,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
6,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
7,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0
8,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.000000,105279.115159,0.000000,0.000000,105279.115159,0.0,0.0


In [41]:
cost.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
0,18500.000000,18500.000000,0.0,0.0,0.000000,18500.000000,18500.000000,0.0,0.0,18500.000000,0.0,0.0
1,1708.333333,20208.333333,0.0,0.0,18500.000000,1708.333333,20208.333333,0.0,0.0,20208.333333,0.0,0.0
2,1708.333333,21916.666667,0.0,0.0,20208.333333,1708.333333,21916.666667,0.0,0.0,21916.666667,0.0,0.0
3,1708.333333,23625.000000,0.0,0.0,21916.666667,1708.333333,23625.000000,0.0,0.0,23625.000000,0.0,0.0
4,1708.333333,25333.333333,0.0,0.0,23625.000000,1708.333333,25333.333333,0.0,0.0,25333.333333,0.0,0.0
5,1708.333333,27041.666667,0.0,0.0,25333.333333,1708.333333,27041.666667,0.0,0.0,27041.666667,0.0,0.0
6,1708.333333,28750.000000,0.0,0.0,27041.666667,1708.333333,28750.000000,0.0,0.0,28750.000000,0.0,0.0
7,1708.333333,30458.333333,0.0,0.0,28750.000000,1708.333333,30458.333333,0.0,0.0,30458.333333,0.0,0.0
8,1708.333333,32166.666667,0.0,0.0,30458.333333,1708.333333,32166.666667,0.0,0.0,32166.666667,0.0,0.0


In [42]:
loanA.amt.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.0,0.0,0.0,0.0,55000.0,0.0,0.0,0.0,0.0,55000.0,0.0,0.0
0,0.0,0.0,55000.0,55000.0,55000.0,0.0,0.0,2000.0,2000.0,53000.0,0.0,53000.0
1,0.0,0.0,0.0,55000.0,53000.0,0.0,0.0,2000.0,4000.0,51000.0,0.0,51000.0
2,0.0,0.0,0.0,55000.0,51000.0,0.0,0.0,2000.0,6000.0,49000.0,0.0,49000.0
3,0.0,0.0,0.0,55000.0,49000.0,0.0,0.0,1000.0,7000.0,48000.0,0.0,48000.0
4,0.0,0.0,0.0,55000.0,48000.0,0.0,0.0,2000.0,9000.0,46000.0,0.0,46000.0
5,0.0,0.0,0.0,55000.0,46000.0,0.0,0.0,2000.0,11000.0,44000.0,0.0,44000.0
6,0.0,0.0,0.0,55000.0,44000.0,0.0,0.0,2000.0,13000.0,42000.0,0.0,42000.0
7,0.0,0.0,0.0,55000.0,42000.0,0.0,0.0,2000.0,15000.0,40000.0,0.0,40000.0
8,0.0,0.0,0.0,55000.0,40000.0,0.0,0.0,2000.0,17000.0,38000.0,0.0,38000.0


In [43]:
loanB.amt.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0
0,0.0,0.0,20000.0,20000.0,20000.0,0.0,0.0,20000.0,20000.0,0.0,0.0,0.0
1,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
2,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
3,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
4,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
5,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
6,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
7,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0
8,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0


In [44]:
loan_fee.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,2875.0,2875.0,0.0,0.0,0.0,2875.0,2875.0,0.0,0.0,2875.0,0.0,0.0
1,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
2,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
3,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
4,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
5,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
6,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
7,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0
8,0.0,2875.0,0.0,0.0,2875.0,0.0,2875.0,0.0,0.0,2875.0,0.0,0.0


In [45]:
loanA.IR.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
1,7.50,7.50,0.0,0.0,0.00,7.50,7.50,0.0,0.0,7.50,0.0,0.0
2,15.00,22.50,0.0,0.0,7.50,15.00,22.50,0.0,0.0,22.50,0.0,0.0
3,22.50,45.00,0.0,0.0,22.50,22.50,45.00,0.0,0.0,45.00,0.0,0.0
4,26.25,71.25,0.0,0.0,45.00,26.25,71.25,0.0,0.0,71.25,0.0,0.0
5,33.75,105.00,0.0,0.0,71.25,33.75,105.00,0.0,0.0,105.00,0.0,0.0
6,41.25,146.25,0.0,0.0,105.00,41.25,146.25,0.0,0.0,146.25,0.0,0.0
7,48.75,195.00,0.0,0.0,146.25,48.75,195.00,0.0,0.0,195.00,0.0,0.0
8,56.25,251.25,0.0,0.0,195.00,56.25,251.25,0.0,0.0,251.25,0.0,0.0


In [46]:
loanB.IR.df

,add_scdd,add_scdd_cum,sub_scdd,sub_scdd_cum,bal_strt,amt_add,amt_add_cum,amt_sub,amt_sub_cum,bal_end,add_rsdl_cum,sub_rsdl_cum
-1,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1,116.666667,116.666667,0.0,0.0,0.000000,116.666667,116.666667,0.0,0.0,116.666667,0.0,0.0
2,116.666667,233.333333,0.0,0.0,116.666667,116.666667,233.333333,0.0,0.0,233.333333,0.0,0.0
3,116.666667,350.000000,0.0,0.0,233.333333,116.666667,350.000000,0.0,0.0,350.000000,0.0,0.0
4,116.666667,466.666667,0.0,0.0,350.000000,116.666667,466.666667,0.0,0.0,466.666667,0.0,0.0
5,116.666667,583.333333,0.0,0.0,466.666667,116.666667,583.333333,0.0,0.0,583.333333,0.0,0.0
6,116.666667,700.000000,0.0,0.0,583.333333,116.666667,700.000000,0.0,0.0,700.000000,0.0,0.0
7,116.666667,816.666667,0.0,0.0,700.000000,116.666667,816.666667,0.0,0.0,816.666667,0.0,0.0
8,116.666667,933.333333,0.0,0.0,816.666667,116.666667,933.333333,0.0,0.0,933.333333,0.0,0.0
